# Solution

## Goal
Use the first 5 strength measurements to propose a **next experimental plan** that is statistically efficient and actionable.

## Problem Statement
Rubber Experts are experimenting with two polymers and two additives (weight %). They ran 5 experiments and measured strength (higher is better).

| Exp | Polymer 1 | Polymer 2 | Additive 1 | Additive 2 | Strength |
| ---: | ---: | ---: | ---: | ---: | ---: |
| 1 | 90 | 0 | 10 | 0 | 16.5 |
| 2 | 0 | 90 | 10 | 0 | 8.6 |
| 3 | 45 | 45 | 0 | 10 | 12.6 |
| 4 | 60 | 30 | 5 | 5 | 18.9 |
| 5 | 30 | 60 | 5 | 5 | 9.8 |

We need to recommend: (1) how many additional experiments to run, (2) what ingredients/proportions to use, and (3) the exact mixes for the next experiments.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Reproducibility
rng = np.random.default_rng(42)

data = pd.DataFrame(
    {
        'exp': [1, 2, 3, 4, 5],
        'polymer_1': [90, 0, 45, 60, 30],
        'polymer_2': [0, 90, 45, 30, 60],
        'additive_1': [10, 10, 0, 5, 5],
        'additive_2': [0, 0, 10, 5, 5],
        'strength': [16.5, 8.6, 12.6, 18.9, 9.8],
    }
)

# Sanity checks on totals
data['total'] = data[['polymer_1', 'polymer_2', 'additive_1', 'additive_2']].sum(axis=1)
data['polymer_total'] = data[['polymer_1', 'polymer_2']].sum(axis=1)
data['additive_total'] = data[['additive_1', 'additive_2']].sum(axis=1)
data

## Key insight: this is a mixture problem (not a standard $2^k$ factorial)
In a classical factorial design, factors vary **independently**.

Here, we have *composition constraints*:
- The four percentages sum to 100%.
- In these experiments, polymer total is fixed at 90% and additive total is fixed at 10%.

So there are only **two independent knobs**:
- $p = \frac{\text{Polymer 1}}{\text{Polymer 1}+\text{Polymer 2}} \in [0,1]$ (polymer blend)
- $a = \frac{\text{Additive 1}}{\text{Additive 1}+\text{Additive 2}} \in [0,1]$ (additive blend)

Given fixed totals (90% polymer, 10% additive), a choice of $(p,a)$ maps back to the actual recipe:
$$\text{Polymer 1}=90p,\ \text{Polymer 2}=90(1-p),\ \text{Additive 1}=10a,\ \text{Additive 2}=10(1-a).$$

In [ ]:
data = data.copy()
data['p'] = data['polymer_1'] / data['polymer_total']
data['a'] = data['additive_1'] / data['additive_total']

cols = ['exp', 'polymer_1', 'polymer_2', 'additive_1', 'additive_2', 'strength', 'p', 'a']
data[cols]

## Quick exploratory look
With only 5 runs, we should be careful about over-interpreting patterns. Still:
- The current best is Exp 4: $(p,a)=(2/3, 1/2)$ with strength 18.9.
- Swapping the polymer blend from $p=2/3$ to $p=1/3$ at $a=1/2$ drops strength dramatically (Exp 4 vs Exp 5).
- We have **no replicates**, so we cannot estimate experimental noise yet.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
sc = ax.scatter(data['p'], data['a'], c=data['strength'], s=140, cmap='viridis', edgecolor='k')
for _, r in data.iterrows():
    ax.annotate('E{}'.format(int(r['exp'])), (r['p'], r['a']), textcoords='offset points', xytext=(6, 6))
ax.set_xlabel('p = Polymer 1 fraction within polymers')
ax.set_ylabel('a = Additive 1 fraction within additives')
ax.set_title('Observed experiments in (p, a) space')
fig.colorbar(sc, ax=ax, label='Strength')
plt.show()

## A first-pass model (don’t overfit)
A reasonable starting model in the 2D space is a linear model with an interaction:
$$\text{strength} = \beta_0 + \beta_p p + \beta_a a + \beta_{pa} (pa) + \varepsilon.$$
,
5

: 
,
: 
,
: {
: 

: [
,
,
,
,

: 
,
: 
,
: {
: 

: [
,

1
,
,

: 
,
: 
,
: {
: 

: [
1
101
,
1
101
,
,
,
1
,
,
,
,
,

: 
,
: 
,
: {
: 

: [
5
,
,
,
,
1
,
1
,
,
,
,

: 
,
: 
,
: {
: 

: [
,
,
,
,
,
,
,
,
,
90
10
,
,
1
4
,
2
0.5
,
3
3
,
4
2

: 
,
: 
,
: {
: 

: [
,
,
,
,
,
,
1
2
,
,
,
,
,
,
,
,